# 結露領域など高相対湿度における接続関数を考える

## 一定以上の相対湿度において、平衡含水率曲線を3次関数で近似する  

### $$ \phi = a h^3 + bh^2 + ch + d $$

- $\phi$：含水率
- $h$：相対湿度

### 変数の定義
- $\psi$：空隙率または毛管飽和含水率
- $g(\phi)$：接続する関数の傾きを表す関数
- $h_c$：関数と接続する際の最小の相対湿度

### 拘束条件
- $\phi(h_{sat})$：飽和時（相対湿度1.0）のときの含水率
- $\phi(h_c)$：関数と接続する際の含水率
- $\frac{\partial \phi(h_c)}{\partial h}$：関数と接続する際の平衡含水率の傾き
- $\frac{\partial \phi(h_{sat})}{\partial h}$：飽和時の平衡含水率の傾き（未知数を解くため）

## 計算方法  
行列計算を用いて連立方程式の解を求める。


### 行列式
$$ A \left(
    \begin{matrix} 
        a \\ 
        b \\
        c \\ 
        d 
    \end{matrix} 
        \right)
    = 
    \left(
    \begin{matrix} 
        \phi(h_{sat}) \\
        \frac{\partial \phi(h_{sat})}{\partial h} \\
        \phi(h_c) \\ 
        \frac{\partial \phi(h_c)}{\partial h}
    \end{matrix} 
        \right)
    
    $$

また、行列Aは以下のように与えられる。

$$ A = \left(
    \begin{matrix} 
        h_{sat}^3 & h_{sat}^2 & h_{sat} & 1.0 \\ 
        3 h_{sat}^2 & 2h_{sat} & 1.0 & 0.0 \\ 
        h_{c}^3 & h_{c}^2 & h_{c} & 1.0 \\ 
        3 h_{c}^2 & 2h_{c} & 1.0 & 0.0
    \end{matrix} 
        \right)$$

In [1]:
function set_A(hsat, hc)
    return ([
        hsat^3      hsat^2  hsat    1.0
        3*hsat^2    2*hsat  1.0     0.0
        hc^3        hc^2    hc      1.0
        3*hc^2      2*hc    1.0     0.0 
        ])
end

set_A (generic function with 1 method)

In [2]:
function cal_abcd(phi_sat, dphi_sat, phi_c, dphi_c, hsat, hc)
    A = set_A(hsat, hc)
    B = [
        phi_sat
        dphi_sat
        phi_c
        dphi_c
    ]
    return A \ B
end

cal_abcd (generic function with 1 method)

In [3]:
include("../cell.jl"); # 多孔質材料の構成要素を表すモジュール
function cal_abcd_by_cell(material_name, hsat, hc)
    cell_c   = Cell( temp = 293.15, miu = convertRH2Miu( temp = 293.15, rh = hc ),   material_name = material_name )
    cell_sat = Cell( temp = 293.15, miu = convertRH2Miu( temp = 293.15, rh = hsat ), material_name = material_name )
    return cal_abcd( psi(cell_sat), 0.0, phi(cell_c), dphi(cell_c), hsat, hc)
end

cal_abcd_by_cell (generic function with 1 method)

In [4]:
a, b, c, d = cal_abcd_by_cell("glass_wool_16K", 1.0, 0.99)

4-element Vector{Float64}:
 -1.9777924016366822e6
  5.903710318859591e6
 -5.874043432809136e6
  1.9481265055862272e6

In [12]:
rht = 0.9999

0.9999

In [13]:
a * rht^3 + b * rht^2 + c * rht + d

0.9897053090389818